In [3]:
from music21 import *
import time
import os
import numpy as np
import itertools as it
from sklearn.model_selection import train_test_split
from sklearn import svm
from scipy.stats import itemfreq
from functools import partial
from pathos.multiprocessing import Pool, cpu_count
import math
# chordified = parsed.chordify().getElementsByClass('Chord')
#type(list(parsed[0])[2]) == key.Key

In [59]:
# Extract notes
def get_notes(song):
    return (str(pitch) for chord in song for pitch in chord.pitches)

# Mockup of the future function
def note_finder(note):
    return note[0]

# Load MIDI data from file
def loader(filename):
    return get_notes(converter.parse(filename))

# Generates a vector with all possible combinations of notes of size 4
#def zero_vector_dictionary(chars):
#    return sum(sum(sum(map(lambda x: map(lambda y: map(lambda z: map(lambda w: [x+y+z+w, 0],
#                                                                     chars), chars), chars), chars), []), []), [])
    #return dict(map(lambda (k, v): (k, 0), zip(combinations, [0]*len(combinations))))

# Generate all sequences of 4 consecutive notes
def get_sequences(data):
    taken = list(it.islice(data, 0, 4))
    while True:
        yield "".join(taken[-4:])
        taken.append(next(data))

# Get file names of type type_ in all the subdirectories of the path
def get_filenames(path, type_ = None):
    if type_ == None:
        return sum(map(lambda walks: (map(lambda name: os.path.join(walks[0], name),
                                                                           walks[2])), os.walk(path)), [])
    else:
        return filter(lambda f: f.endswith(type_), sum(map(lambda walks: (map(lambda name: os.path.join(walks[0], name),
                                                                           walks[2])), os.walk(path)), []))

# Returns index of the note quartet in the vector
def notestoindex(chars):
    return reduce(lambda x, y: 7*x+y, [0] + map(lambda x: "ABCDEFG".index(x), chars))

# Build vector of ocurrences of note quartets
def build_vector(sequence):
    vector = [0] * 7**4
    sequence = list(sequence)
    freqs = {notestoindex(x[0]) : int(x[1]) for x in itemfreq(sequence)}
    for (k, v) in freqs.iteritems():
        vector[k] = v
    return vector

def vector_from_file(path):
    sequence = get_sequences((e[0] for e in loader(path)))
    return build_vector(sequence)

# Build x vectors from a given path and assign y = class_ to them
def vectors_from_files(path, class_, count):
    names = get_filenames(path)[:count]
    vectors = pool.map(vector_from_file, names)
    return vectors, [class_] * len(vectors)

# Build vectors without pool works much slower
def vectors_from_files_no_pool(path, class_, count):
    X = map(lambda name: build_vector(get_sequences(map(lambda note: map(note_finder, note), loader(name)))), 
            get_filenames(path)[:count])
    return X, [class_]*len(X)

In [19]:
path_to_class0 = './50s/'
path_to_class1 = './60s/'
# Renaming 50s and 60s .kar to .mid
map1 = map(lambda name: os.rename(name, name.replace(".kar", ".mid").replace(".KAR", ".mid")), 
           get_filenames(path_to_class0))
map2 = map(lambda name: os.rename(name, name.replace(".kar", ".mid").replace(".KAR", ".mid")), 
           get_filenames(path_to_class1))

# Warming up the processors and creating a pool of half of the processors in the system
pool = Pool(processes = cpu_count())
res = pool.map(lambda x: x+1, range(cpu_count()))

# Identifying the smallest directory in terms of number of midis
midi_count = min(len(get_filenames(path_to_class0)), len(get_filenames(path_to_class1)))
midi_count

76

In [43]:
path_to_class0 = './50s/'
path_to_class1 = './60s/'
start_time = time.time()
vectors0 = list(vectors_from_files(path_to_class0, 0, midi_count))
print "Class 0 is done"
print("--- %s seconds ---" % (time.time() - start_time))
vectors1 = list(vectors_from_files(path_to_class1, 1, midi_count))
print "Class 1 is done"
print("--- %s seconds ---" % (time.time() - start_time))

Class 0 is done
--- 9.81566500664 seconds ---
Class 1 is done
--- 18.7039480209 seconds ---


In [143]:
# Merging the vectors together and extracting the attributes and the label
X = vectors0[0]+vectors1[0]
Y = vectors0[1]+vectors1[1]

# Splitting the data into training and testing subsets 90 to 10 %
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1, random_state = 25)

# Splitting the training subset into training and validating subsets 80 to 20 %
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, test_size=0.2, random_state = 25)

In [144]:
print len(X_train), len(Y_train)
print len(X_valid), len(Y_valid)
print len(X_test), len(Y_test)

108 108
28 28
16 16


In [183]:
# Fitting the parameter C for the SVM model on the training / validation sets
clf = svm.SVC(kernel = 'poly', degree = 2, C=0.9)
clf.fit(X_train, Y_train)
print "We successfully predict the {} percent of training data".format((Y_train==clf.predict(X_train)).sum()*100/len(Y_train))
print "We successfully predict the {} percent of test data".format((Y_test==clf.predict(X_test)).sum()*100/len(Y_test))

We successfully predict the 100 percent of training data
We successfully predict the 68 percent of test data


In [184]:
# Function to train SVM on training set and to predict the validation set
def trainpredict(degree, C):
    clf = svm.SVC(kernel = 'poly', degree = degree, C = C) 
    clf.fit(X_train, Y_train)
    return (Y_valid == clf.predict(X_valid)).sum()*100/len(Y_valid), degree, C

# Function to fit poly SVMs of degrees 1 to 4 each with Cs from the list
# [math.exp(i) for i in np.linspace(-10,2*x,200)]
def fitDegree_C():
    return map(lambda degree: map(trainpredict, [degree]*len(np.linspace(-10, 2*degree, 200)),
                            map(lambda c: math.exp(c), np.linspace(-10, 2*degree, 200))), range(1, 5)) 
# Each of the sublists correspond to a degree (1 to 4) and has tuples with (accuracy, degree, C)
fitted = fitDegree_C()

In [171]:
# print 1.0*np.count_nonzero(map(lambda i: X[i] == vectors0[0][i], range(0, len(vectors0[0]))))/len(vectors0[0])
# print 1.0*np.count_nonzero(map(lambda i: X[i+76] == vectors1[0][i], range(0, len(vectors1[0]))))/len(vectors1[0])
# print 1.0*np.count_nonzero(map(lambda i: Y[i] == vectors0[1][i], range(0, len(vectors0[1]))))/len(vectors0[1])
# print 1.0*np.count_nonzero(map(lambda i: Y[i+76] == vectors1[1][i], range(0, len(vectors1[1]))))/len(vectors1[1])
# #np.count_nonzero([True, True])

1.0
1.0
1.0
1.0


In [175]:
flatten = sum(fitted, [])
# Looking for the optimal parameters
max(enumerate(flatten), key = lambda x: x[1])

(228, (53, 2, 0.00032549913500013486))